In [4]:
from src.signal_generator import SignalGenerator
from src.metrics import Metric
from src.data_type import Measurement
import matplotlib.pyplot as plt
from src.data_compressor.other import CompressNTHS
from src.data_compressor.other import CompressMinMax
from src.data_compressor.other import CompressPWP
from src.data_compressor.pip import CompressPIP_ED
from src.data_compressor.pip import CompressPIP_PD
from src.data_compressor.pip import CompressPIP_VD
from src.data_compressor.paa import CompressPAA
from src.data_compressor.paa import CompressPAAVI
from src.data_compressor.paa import CompressByChunk
from src.data_compressor.pla import CompressAPCADFT
from src.data_compressor.pla import CompressAPCAFFT
from src.data_compressor.pla import CompressSTC
from src.data_compressor.pla import CompressHigherDeriveration
from typing import List, Dict
import json

In [2]:
measurements = [
  Measurement(5, 1000),
  Measurement(3, 1100),
  Measurement(2, 1200),
]

In [ ]:
class CBOR_Serializer:
  def serialize(measurements: List[Measurement], a: int, b: int, c: int):
    output = bytearray()
    output.append(0xA1)
    return output

cbor_parser = CBOR_Serializer()
print(cbor_parser.serialize(measurements))

In [11]:
class SenML_JSON_Serializer:
  def serialize(self, measurements: List[Measurement], bn_value: str, n_value: str) -> str:
    output = []
    values = [m.value for m in measurements]
    timestamps = [m.timestamp for m in measurements]
    data_size = len(values)
    index = data_size - 1
    base_time = timestamps[index]
    output.append({
      'bn': bn_value,
      'n': n_value,
      'v': values[index],
      'bt': timestamps[index],
      't': 0
    })
    index -= 1
    while index > -1:
      output.append({
        'n': n_value,
        'v': values[index],
        't': timestamps[index] - base_time,
      })
      index -= 1
    return json.dumps(output)

senml_json_serializer = SenML_JSON_Serializer()
print(senml_json_serializer.serialize(measurements, '/72/', '1/2'))

[{"bn": "/72/", "n": "1/2", "v": 2, "bt": 1200, "t": 0}, {"n": "1/2", "v": 3, "t": -100}, {"n": "1/2", "v": 5, "t": -200}]
